In [1]:
# for package auto reload
%load_ext autoreload
%autoreload 2

# for better rendering of plots in jupyter notebook
%matplotlib inline 

In [2]:
# base modules
import os
import sys
import copy 

# Import needed packages
#To manipulate data
import numpy as np
import pandas as pd

#To display graphs
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import tree
import graphviz



#To run models
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, BaggingRegressor
from sklearn.svm import LinearSVC
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn import preprocessing
# custom module
sys.path.insert(0, os.path.join('/Users/bcoquard/Python lesson/Notebooks/advance ML-Course-2022-T1-main', 'src'))

# custom module
from emlyon.utils import *



In [3]:
#load the data sets
df_raw=pd.read_csv('/Users/bcoquard/Python lesson/Notebooks/Kaggle Competition/titanic/train.csv')
df_test=pd.read_csv('/Users/bcoquard/Python lesson/Notebooks/Kaggle Competition/titanic/test.csv')


In [4]:
df_raw

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [5]:
df_test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [6]:
df_raw.info(),
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass  

In [7]:
df_raw.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [8]:
df_raw.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [9]:
df_test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

>> Looking at the datas for the Train Dataset ( Df_raw)
- So we could see that  the column " Cabin" is at 77% composed of NaN values, however this might be an important feature to help predict if the passager has survided. As a matter of fact, Cabins location on the boat depended on the level of cabin and this might have played a role in the change to survive. As we have the price of all the tickets we might be able to predict the cabin level. So we will keep the column and categorize it.

- Almost 20% of Age column is Nan which is another critical feature that might have played a role in the chance to survive. Let see if we have other way to predict the age range based on other features or on class age variance. As age column only have 20% Nan we might be looking at replacing Nan values by mean. It is an option we will use at first.

- Columns "Name" and "sex" are objects. We might be willing to convert "sex" into a categorical column.
- Name is not really interesting has it is hard to convert , but inthe name , we have the notion of Mister, Miss or Misses which can be easily categorized and might play another role to increase our precision while predicting. Let's keep it in mind. For the first try we will probably  drop the column.

- For the column "Embarked" composed of only 2 Nan we will replace the Nan Values with "NA".

Let's have a look at the rows for which "Age" is a Nan Value.


In [10]:
#Display all the rows with Nan values for the column 'Age'
df_raw[df_raw['Age'].isnull().T]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
17,18,1,2,"Williams, Mr. Charles Eugene",male,NaN,0,0,244373,13.0000,NaN,S
19,20,1,3,"Masselmani, Mrs. Fatima",female,NaN,0,0,2649,7.2250,NaN,C
26,27,0,3,"Emir, Mr. Farred Chehab",male,NaN,0,0,2631,7.2250,NaN,C
28,29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,NaN,0,0,330959,7.8792,NaN,Q
...,...,...,...,...,...,...,...,...,...,...,...,...
859,860,0,3,"Razi, Mr. Raihed",male,NaN,0,0,2629,7.2292,NaN,C
863,864,0,3,"Sage, Miss. Dorothy Edith ""Dolly""",female,NaN,8,2,CA. 2343,69.5500,NaN,S
868,869,0,3,"van Melkebeke, Mr. Philemon",male,NaN,0,0,345777,9.5000,NaN,S
878,879,0,3,"Laleff, Mr. Kristo",male,NaN,0,0,349217,7.8958,NaN,S


In [11]:
#Display all the rows without Nan values for the column 'Cabin'
df_raw[df_raw['Cabin'].notnull().T]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S
11,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S
...,...,...,...,...,...,...,...,...,...,...,...,...
871,872,1,1,"Beckwith, Mrs. Richard Leonard (Sallie Monypeny)",female,47.0,1,1,11751,52.5542,D35,S
872,873,0,1,"Carlsson, Mr. Frans Olof",male,33.0,0,0,695,5.0000,B51 B53 B55,S
879,880,1,1,"Potter, Mrs. Thomas Jr (Lily Alexenia Wilson)",female,56.0,0,1,11767,83.1583,C50,C
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S


In [12]:
df_raw['Cabin'].sort_values(ascending=True)

583    A10
475    A14
556    A16
284    A19
599    A20
      ... 
884    NaN
885    NaN
886    NaN
888    NaN
890    NaN
Name: Cabin, Length: 891, dtype: object

> Ordering the values of the column Cabin, we can note :
- Cabin numbers all start with a letter that goers from A to G included and there is a single row with a T.
- Some values are composed of multiple room numbers which belongs to the wsame family siblings.Since the rooms are located closed to each others it doesn't have a huge impact if we only keep one location and not all.

In [13]:
#Display all the rows with Nan values for the column 'Age'
df_raw[df_raw['Embarked'].isnull().T]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN


> Interesting to note that the 2 women for which we don't have the harbor where they embarked ,  where staying in the same room , with the same ticket number and have both survived.

## Let's clean the missing values

In [14]:
#In df_raw , Let's replace Nan values in "Age" with median, "Embarked" with 'U' for Unknow, and "Cabin with Unknown"
#So that we can cnumericalize all columns we want to keep.
df_raw['Age'].fillna(df_raw['Age'].median(), inplace=True)
df_raw['Embarked'].fillna('U', inplace=True)
df_raw['Cabin'].fillna('Uknown', inplace=True)

In [15]:
#In df_test , Let's replace Nan values in "Age" and "Fare" with median and "Cabin with Unknown"
#So that we can cnumericalize all columns we want to keep.
df_test['Age'].fillna(df_test['Age'].median(), inplace=True)
df_test['Fare'].fillna(df_test['Fare'].median(), inplace=True)
df_test['Cabin'].fillna('Uknown', inplace=True)

In [16]:
df_raw

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,Uknown,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,Uknown,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,Uknown,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,Uknown,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,28.0,1,2,W./C. 6607,23.4500,Uknown,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [17]:
df_raw.isnull().sum() # We have cleaned all Nan values.

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [18]:
df_test.isnull().sum() # We have cleaned all Nan values.

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

## Dropping some columns and numericalize all columns kept.

In [19]:
#dropping columns Ticket, PassengerId, Name. The row indexes are the passager Id.
df_raw=df_raw.drop(['Ticket','Name'], axis=1) 
df_test=df_test.drop(['Ticket','Name'], axis=1) 
df_raw

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,1,0,3,male,22.0,1,0,7.2500,Uknown,S
1,2,1,1,female,38.0,1,0,71.2833,C85,C
2,3,1,3,female,26.0,0,0,7.9250,Uknown,S
3,4,1,1,female,35.0,1,0,53.1000,C123,S
4,5,0,3,male,35.0,0,0,8.0500,Uknown,S
...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,male,27.0,0,0,13.0000,Uknown,S
887,888,1,1,female,19.0,0,0,30.0000,B42,S
888,889,0,3,female,28.0,1,2,23.4500,Uknown,S
889,890,1,1,male,26.0,0,0,30.0000,C148,C


In [20]:
#Let's keep only the letter in the Cabin Columns values
df_raw['Cabin'] = [x[0] for x in df_raw['Cabin']]
df_test['Cabin'] = [x[0] for x in df_test['Cabin']]

In [21]:
df_raw

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,1,0,3,male,22.0,1,0,7.2500,U,S
1,2,1,1,female,38.0,1,0,71.2833,C,C
2,3,1,3,female,26.0,0,0,7.9250,U,S
3,4,1,1,female,35.0,1,0,53.1000,C,S
4,5,0,3,male,35.0,0,0,8.0500,U,S
...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,male,27.0,0,0,13.0000,U,S
887,888,1,1,female,19.0,0,0,30.0000,B,S
888,889,0,3,female,28.0,1,2,23.4500,U,S
889,890,1,1,male,26.0,0,0,30.0000,C,C


In [22]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Sex          891 non-null    object 
 4   Age          891 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Fare         891 non-null    float64
 8   Cabin        891 non-null    object 
 9   Embarked     891 non-null    object 
dtypes: float64(2), int64(5), object(3)
memory usage: 69.7+ KB


In [23]:
#let's convert columns Sex, Embarked, Cabin  into categorical and numericalize them
le= preprocessing.LabelEncoder() #to run the label encoder you need to make sure you have not changed the type to category first
cols=["Sex","Embarked","Cabin"]
for col in cols:
    df_raw[col]=le.fit_transform(df_raw[col])
    df_test[col]=le.transform(df_test[col])
    print(le.classes_)
df_raw.head(5)

['female' 'male']
['C' 'Q' 'S' 'U']
['A' 'B' 'C' 'D' 'E' 'F' 'G' 'T' 'U']


,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,1,0,3,1,22.0,1,0,7.2500,8,2
1,2,1,1,0,38.0,1,0,71.2833,2,0
2,3,1,3,0,26.0,0,0,7.9250,8,2
3,4,1,1,0,35.0,1,0,53.1000,2,2
4,5,0,3,1,35.0,0,0,8.0500,8,2


### Columns "Sex, Cabin, Embarked" have been changed to Integer type and numericalized.
> For column " Sex" : 1= female and 2= male
> For column " Cabin" : 1=A ,2= B, 3=C,4=D,5=E,6=F,7=G, 8=T, 9=U
> For column " Embarked" : 1= C and 2= Q, 3= S, 4=U

In [24]:
#Let's define X and Y our Input and Target for bothe dataset
ytrain = df_raw['Survived']
xtrain = df_raw.drop('Survived', axis=1)
xtest=df_test

In [25]:
xtrain

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,1,3,1,22.0,1,0,7.2500,8,2
1,2,1,0,38.0,1,0,71.2833,2,0
2,3,3,0,26.0,0,0,7.9250,8,2
3,4,1,0,35.0,1,0,53.1000,2,2
4,5,3,1,35.0,0,0,8.0500,8,2
...,...,...,...,...,...,...,...,...,...
886,887,2,1,27.0,0,0,13.0000,8,2
887,888,1,0,19.0,0,0,30.0000,1,2
888,889,3,0,28.0,1,2,23.4500,8,2
889,890,1,1,26.0,0,0,30.0000,2,0


In [26]:
ytrain

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [27]:
xtest

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,892,3,1,34.5,0,0,7.8292,8,1
1,893,3,0,47.0,1,0,7.0000,8,2
2,894,2,1,62.0,0,0,9.6875,8,1
3,895,3,1,27.0,0,0,8.6625,8,2
4,896,3,0,22.0,1,1,12.2875,8,2
...,...,...,...,...,...,...,...,...,...
413,1305,3,1,27.0,0,0,8.0500,8,2
414,1306,1,0,39.0,0,0,108.9000,2,0
415,1307,3,1,38.5,0,0,7.2500,8,2
416,1308,3,1,27.0,0,0,8.0500,8,2


# Split the Train dataset into x and y and train the model

In [28]:
#let's use Random Forest Classifier and fit
model=RandomForestClassifier(
    n_estimators = 100, 
    class_weight = None, # classifier specific
    criterion = 'gini',  # classifier specific
    max_depth = None, 
    min_samples_split = 2, 
    min_samples_leaf = 1, 
    min_weight_fraction_leaf = 0.0,  
    max_leaf_nodes = None, 
    min_impurity_decrease = 0.0, 
    ccp_alpha = 0.0, 
    random_state = 42, 
    bootstrap = True, 
    oob_score = True, 
    max_samples = None,
    warm_start = False, 
    n_jobs = -1, 
    verbose = 0, 
)
%time model.fit(xtrain, ytrain)

y_pred = model.predict(xtrain)
print(model.score(xtrain,ytrain))
acc_rmodel = round(model.score(xtrain, ytrain) * 100, 2)
print(acc_rmodel)
print(model.oob_score_)    

CPU times: user 154 ms, sys: 77.6 ms, total: 231 ms
Wall time: 135 ms
1.0
100.0
0.8181818181818182


### Let's use Grid Search CV to optimize the paramters

In [29]:
# Let's use Grid Search CV to 
param_grid = [
    {'n_estimators': [10,200,500,1000],
     'max_depth': [10, 20, None],
     'min_samples_split': [2, 10],
     'max_samples': [0.4, 0.75, 1.],
     'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
    },
]


In [30]:
tuned_model = GridSearchCV(
    estimator = model, 
    param_grid = param_grid, 
    scoring = None, # uses estimator's default score method so R2
    n_jobs = -1, 
    refit = True, # keep a fitted version of the overall best model
    cv = 5, 
    return_train_score = True,
    verbose = 2,
)

In [31]:
%time tuned_model.fit(xtrain, ytrain) 

Fitting 5 folds for each of 720 candidates, totalling 3600 fits


/Users/bcoquard/opt/anaconda3/envs/mlcourse/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/bcoquard/opt/anaconda3/envs/mlcourse/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/bcoquard/opt/anaconda3/envs/mlcourse/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly 

/Users/bcoquard/opt/anaconda3/envs/mlcourse/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/bcoquard/opt/anaconda3/envs/mlcourse/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/bcoquard/opt/anaconda3/envs/mlcourse/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly 

/Users/bcoquard/opt/anaconda3/envs/mlcourse/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/bcoquard/opt/anaconda3/envs/mlcourse/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/bcoquard/opt/anaconda3/envs/mlcourse/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly 

/Users/bcoquard/opt/anaconda3/envs/mlcourse/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/bcoquard/opt/anaconda3/envs/mlcourse/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:586: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(
/Users/bcoquard/opt/anaconda3/envs/mlcourse/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:586: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(
/Users/bcoquard/opt/anaconda3/envs/mlcourse/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:586: UserWarning: Some input

...

[CV] END criterion=entropy, max_depth=7, max_features=auto, max_samples=0.4, min_samples_split=2, n_estimators=200; total time=   0.7s
[CV] END criterion=entropy, max_depth=6, max_features=auto, max_samples=1.0, min_samples_split=2, n_estimators=500; total time=   1.2s
[CV] END criterion=entropy, max_depth=6, max_features=auto, max_samples=1.0, min_samples_split=10, n_estimators=200; total time=   0.9s
[CV] END criterion=entropy, max_depth=6, max_features=auto, max_samples=1.0, min_samples_split=10, n_estimators=200; total time=   0.6s
[CV] END criterion=entropy, max_depth=6, max_features=auto, max_samples=1.0, min_samples_split=10, n_estimators=500; total time=   1.1s
[CV] END criterion=entropy, max_depth=6, max_features=sqrt, max_samples=0.4, min_samples_split=2, n_estimators=10; total time=   0.1s
[CV] END criterion=entropy, max_depth=6, max_features=sqrt, max_samples=0.4, min_samples_split=2, n_estimators=200; total time=   0.7s
[CV] END criterion=entropy, max_depth=6, max_features

[CV] END criterion=entropy, max_depth=7, max_features=sqrt, max_samples=1.0, min_samples_split=2, n_estimators=500; total time=   1.4s
[CV] END criterion=entropy, max_depth=6, max_features=sqrt, max_samples=0.75, min_samples_split=2, n_estimators=10; total time=   0.0s
[CV] END criterion=entropy, max_depth=6, max_features=sqrt, max_samples=0.75, min_samples_split=2, n_estimators=10; total time=   0.1s
[CV] END criterion=entropy, max_depth=6, max_features=sqrt, max_samples=0.75, min_samples_split=2, n_estimators=200; total time=   0.6s
[CV] END criterion=entropy, max_depth=6, max_features=sqrt, max_samples=0.75, min_samples_split=2, n_estimators=200; total time=   0.7s
[CV] END criterion=entropy, max_depth=6, max_features=sqrt, max_samples=0.75, min_samples_split=2, n_estimators=500; total time=   1.4s
[CV] END criterion=entropy, max_depth=6, max_features=sqrt, max_samples=0.75, min_samples_split=10, n_estimators=10; total time=   0.1s
[CV] END criterion=entropy, max_depth=6, max_featur

/Users/bcoquard/opt/anaconda3/envs/mlcourse/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


CPU times: user 4.77 s, sys: 721 ms, total: 5.49 s
Wall time: 9min 1s


GridSearchCV(cv=5,
             estimator=RandomForestClassifier(n_jobs=-1, oob_score=True,
                                              random_state=42),
             n_jobs=-1,
             param_grid=[{'criterion': ['gini', 'entropy'],
                          'max_depth': [4, 5, 6, 7, 8],
                          'max_features': ['auto', 'sqrt', 'log2'],
                          'max_samples': [0.4, 0.75, 1.0],
                          'min_samples_split': [2, 10],
                          'n_estimators': [10, 200, 500, 1000]}],
             return_train_score=True, verbose=2)

In [32]:
%time best_model = tuned_model.best_estimator_
best_model

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 3.81 µs


RandomForestClassifier(max_depth=8, max_features='auto', max_samples=0.75,
                       min_samples_split=10, n_estimators=500, n_jobs=-1,
                       oob_score=True, random_state=42)

In [33]:
# let run for score suing those optimized parameters
model_op = RandomForestClassifier(n_estimators = 1000, 
                               max_depth = 8,
                               max_samples= 1,
                               max_features= 'auto',
                               n_jobs = -1,
                               random_state = 42)

#let's use Random Forest Classifier and fit
model_op=RandomForestClassifier(
    n_estimators = 500, 
    class_weight = None, # classifier specific
    criterion = 'gini',  # classifier specific
    max_depth = 8, 
    max_features='auto',
    min_samples_split = 2, 
    min_samples_leaf = 1, 
    min_weight_fraction_leaf = 0.0,  
    max_leaf_nodes = None, 
    min_impurity_decrease = 0.0, 
    ccp_alpha = 0.0, 
    random_state = 42, 
    bootstrap = True, 
    oob_score = True, 
    max_samples = 0.75,
    warm_start = False, 
    n_jobs = -1, 
    verbose = 0, 
)
%time model_op.fit(xtrain, ytrain)


y_pred = model_op.predict(xtrain)
print(model_op.score(xtrain,ytrain))
acc_rmodel = round(model_op.score(xtrain, ytrain) * 100, 2)
print(acc_rmodel)
print(model_op.oob_score_)    

/Users/bcoquard/opt/anaconda3/envs/mlcourse/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


CPU times: user 665 ms, sys: 288 ms, total: 953 ms
Wall time: 555 ms
0.920314253647587
92.03
0.8305274971941639


In [34]:
predictions = model_op.predict(xtest)

In [36]:
submission = pd.DataFrame({
        "PassengerId": df_test["PassengerId"],
        "Survived": predictions
    })

In [37]:
submission

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [38]:
submission.to_csv('submissionrf.csv', index=False)


[CV] END criterion=entropy, max_depth=7, max_features=sqrt, max_samples=0.4, min_samples_split=2, n_estimators=500; total time=   1.4s
[CV] END criterion=entropy, max_depth=7, max_features=sqrt, max_samples=0.4, min_samples_split=10, n_estimators=200; total time=   0.7s
[CV] END criterion=entropy, max_depth=7, max_features=sqrt, max_samples=0.4, min_samples_split=10, n_estimators=200; total time=   0.4s
[CV] END criterion=entropy, max_depth=7, max_features=sqrt, max_samples=0.4, min_samples_split=10, n_estimators=500; total time=   1.3s
[CV] END criterion=entropy, max_depth=7, max_features=sqrt, max_samples=0.75, min_samples_split=2, n_estimators=10; total time=   0.0s
[CV] END criterion=entropy, max_depth=7, max_features=sqrt, max_samples=0.75, min_samples_split=2, n_estimators=10; total time=   0.0s
[CV] END criterion=entropy, max_depth=7, max_features=sqrt, max_samples=0.75, min_samples_split=2, n_estimators=10; total time=   0.1s
[CV] END criterion=entropy, max_depth=7, max_featur

/Users/bcoquard/opt/anaconda3/envs/mlcourse/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/bcoquard/opt/anaconda3/envs/mlcourse/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/bcoquard/opt/anaconda3/envs/mlcourse/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly 